In [4]:
import jsonlines
from transformers import AutoTokenizer, LlamaForCausalLM, AutoConfig
import torch
from datasets import load_dataset
import datasets
import os
from tqdm.notebook import tqdm
import json

In [1]:
data_names = ["FinGPT/fingpt-sentiment-train", "FinGPT/fingpt-finred", "FinGPT/fingpt-headline", "FinGPT/fingpt-ner", "FinGPT/fingpt-fiqa_qa"]
all_data = load_dataset(data_names[0])['train']
for x in data_names[1:]:
    dataset = load_dataset(x)
    dataset = dataset['train']
    all_data = datasets.concatenate_datasets([all_data, dataset])



/home/wangd12/miniconda3/envs/mamba/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
all_data

Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 204112
})

In [14]:
# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

def process_data(example):
  context = f"Instruction: {example['instruction']}\nInput: {example['input']}\nAnswer: "
  target = example['output']
  return {'context': context, 'target': target}

# Process the entire dataset
processed_data = all_data.map(process_data)

# Save the processed data to a jsonl file
with jsonlines.open('fingpt_data_train.jsonl', 'w') as writer:
  writer.write_all(processed_data)


In [14]:
model_name = "meta-llama/Llama-3.1-8B-Instruct"
jsonl_path = "fingpt_data_train.jsonl"
save_path = 'fingpt_data_train_token'
max_seq_length = 512
skip_overlength = True

def preprocess(tokenizer, config, example, max_seq_length):
    prompt = example["context"]
    target = example["target"]
    prompt_ids = tokenizer.encode(prompt, max_length=max_seq_length, truncation=True)
    target_ids = tokenizer.encode(
        target,
        max_length=max_seq_length,
        truncation=True,
        add_special_tokens=False)
    input_ids = prompt_ids + target_ids + [config.eos_token_id[0]]
    return {"input_ids": input_ids, "seq_len": len(prompt_ids)}

def read_jsonl(path, max_seq_length, skip_overlength=False):
    tokenizer = AutoTokenizer.from_pretrained(
        model_name, trust_remote_code=True)
    config = AutoConfig.from_pretrained(
        model_name, trust_remote_code=True, device_map='auto')
    with open(path, "r") as f:
        for line in tqdm(f.readlines()):
            example = json.loads(line)
            feature = preprocess(tokenizer, config, example, max_seq_length)
            if skip_overlength and len(feature["input_ids"]) > max_seq_length:
                continue
            feature["input_ids"] = feature["input_ids"][:max_seq_length]
            yield feature

dataset = datasets.Dataset.from_generator(
    lambda: read_jsonl(jsonl_path, max_seq_length, skip_overlength), num_proc = 64
    )
dataset.save_to_disk(save_path)

Setting num_proc from 64 back to 1 for the train split to disable multiprocessing as it only contains one shard.


Generating train split: 0 examples [00:00, ? examples/s]

  0%|          | 0/204112 [00:00<?, ?it/s]

Saving the dataset (0/1 shards):   0%|          | 0/201947 [00:00<?, ? examples/s]